In [1]:
import pandas as pd
from matplotlib import pyplot
from IPython.core.display import display
%matplotlib inline

In [2]:
a_df = pd.read_csv("data/train/train_A.tsv", sep='\t')
b_df = pd.read_csv("data/train/train_B.tsv", sep='\t')
c_df = pd.read_csv("data/train/train_C.tsv", sep='\t')
d_df = pd.read_csv("data/train/train_D.tsv", sep='\t')
a_dic = {"label":"A", "category":"人材", "df":a_df}
b_dic = {"label":"B", "category":"旅行", "df":b_df}
c_dic = {"label":"C", "category":"不動産", "df":c_df}
d_dic = {"label":"D", "category":"アパレル", "df":d_df}
data_list = [a_dic, b_dic, c_dic, d_dic]

## データ数

In [40]:
for data in data_list:
    print(data["label"], data["category"])
    print("{0:<5}{1:>7}{2:>2}".format("row",len(data["df"].index), "行"))
    print("{0:<5}{1:>7}{2:>2}".format("user", len(set(data["df"].user_id)), "人"))
    print("{0:<5}{1:>7}{2:>2}".format("item", len(set(data["df"].product_id)), "個"))
    tmp = data["df"]
    print("{0:<5}{1:>7}{2:>2}".format("cv", len(tmp[tmp["event_type"] == 3]), "個"))
    print("{0:<5}{1:>7}{2:>2}".format("cl", len(tmp[tmp["event_type"] == 2]), "回"))
    print("{0:<5}{1:>7}{2:>2}".format("pd", len(tmp[tmp["event_type"] == 1]), "回"))
    print("{0:<5}{1:>7}{2:>2}".format("ca", len(tmp[tmp["event_type"] == 0]), "個"))
    print()

A 人材
row  3376493 行
user   58658 人
item   13866 個
cv     74561 個
cl     33414 回
pd   1981780 回
ca   1286738 個

B 旅行
row  1113425 行
user   23843 人
item   16894 個
cv     21710 個
cl      3439 回
pd   1035637 回
ca     52639 個

C 不動産
row  1666650 行
user   43271 人
item  313609 個
cv      1358 個
cl     14670 回
pd   1650622 回
ca         0 個

D アパレル
row  3572842 行
user  108607 人
item  791822 個
cv       781 個
cl         0 回
pd   1049690 回
ca   2522371 個



## ユーザーは商品をどのくらい買っているのか？

A～Dで共通して、
* 購入経験のあるほとんどのユーザーが１ヵ月間で１、２個しか購入していない。  
→購入上位者のデータがノイズになる可能性がある（特にA,B）  
* ユーザー数に対し、購入経験のあるユーザーが少ない。(特にC,Dで顕著)

** 次のアクション：購入経験の多いユーザーとその他の購入経験のあるユーザー、購入経験のないユーザーを分けてデータ分析してみる。**
* 購入経験の多いユーザー  
どのような流れ（閲覧数・カート）で商品購入しているのかが分かれば、４月末のデータを用いて５月頭の行動を予測できる
* 購入経験のないユーザー  
閲覧等はしているのか？　していなかったら、買う可能性はほぼゼロ？？？（ノンアクティブユーザー）

In [87]:
display_df = pd.DataFrame()
for data in data_list:
    df = data["df"]
    df = df[df["event_type"] == 3]
    count_df = pd.DataFrame(df.groupby("user_id").size(), columns=[data["label"] + "_cv"])
    if display_df.empty:
        display_df = count_df.describe().T
    else:
        display_df = pd.concat([display_df, count_df.describe().T])
display(display_df)

,count,mean,std,min,25%,50%,75%,max
A_cv,17865.0,4.173580,7.615173,1.0,1.0,2.0,4.0,213.0
B_cv,7957.0,2.728415,4.740184,1.0,1.0,2.0,3.0,302.0
C_cv,669.0,2.029895,1.938192,1.0,1.0,1.0,2.0,15.0
D_cv,383.0,2.039164,2.765917,1.0,1.0,1.0,2.0,39.0


## カテゴリーごとにユーザーの行動の傾向に違いはないか？

** 結論：カテゴリーごとにユーザーの行動の違いが存在した。 **

### A（人材）について
* 閲覧数＞＞カートに入れる数＞購入数
* 閲覧数とカートに入れる数の分散が大きい
    * 上位に２,３桁違うほどに閲覧/カートに入れることをする人が存在するため？

### B（旅行）について
* 閲覧数＞＞カートに入れる数＞購入数
* 閲覧数の分散が大きい  
    * ただ旅行行きたいなあと思っている人と、本気で旅行に行こうとしている人に分かれるため？
    * 上位に３桁違うほどに閲覧している人が存在するため？

### C（不動産）について
* カートは存在しない  
* 閲覧数＞＞＞＞＞購入数  
不動産は大きい買い物のため、よく吟味しているものと思われる。  
    →**仮説：閲覧は多いが、まだ購入されていないものが新しく購入されるのではないか？**
* 閲覧数の標準偏差が大きい  
ただ家をなんとなく見ている人と、本気で買おうとしている人とに分かれるため？


### D（アパレル）について
* 広告クリックは存在しない。
* カートに入れる数＝閲覧数＞＞＞＞＞購入数  
ウィンドウショッピングが非常に多い？
* カートに入れる数の標準偏差が大きい  
とりあえず商品詳細を見ずにカートに入れる人とすぐにはカートに入れないひとがそれぞれある程度存在するため。


In [77]:
display_df = pd.DataFrame()
events = ["ca","pd","cl","cv"]
for data in data_list:
    for i, event in enumerate(events):
        if ((data["label"] == "C") and (event == "ca")) or ((data["label"] == "D") and (event == "cl")):
            continue
        df = data["df"]
        df = df[df["event_type"] == i]
        count_df = pd.DataFrame(df.groupby("user_id").size(), columns=[data["label"] + "_" + event])
        if display_df.empty:
            display_df = count_df.describe().T
        else:
            display_df = pd.concat([display_df, count_df.describe().T])
    display(display_df)
    display_df = pd.DataFrame()
#countはその行動経験のあるユーザー数を示している。

,count,mean,std,min,25%,50%,75%,max
A_ca,36619.0,35.138535,97.509113,1.0,8.0,10.0,30.0,8168.0
A_pd,50829.0,38.989160,93.355619,1.0,14.0,23.0,41.0,11987.0
A_cl,8667.0,3.855313,5.081775,1.0,1.0,2.0,4.0,103.0
A_cv,17865.0,4.173580,7.615173,1.0,1.0,2.0,4.0,213.0


,count,mean,std,min,25%,50%,75%,max
B_ca,9362.0,5.622623,6.826006,1.0,2.0,4.0,7.0,224.0
B_pd,23837.0,43.446617,128.110126,2.0,21.0,30.0,47.0,12335.0
B_cl,1498.0,2.295728,4.322503,1.0,1.0,1.0,2.0,88.0
B_cv,7957.0,2.728415,4.740184,1.0,1.0,2.0,3.0,302.0


,count,mean,std,min,25%,50%,75%,max
C_pd,43270.0,38.147030,56.111389,5.0,16.0,24.0,41.0,2772.0
C_cl,6061.0,2.420393,2.366461,1.0,1.0,2.0,3.0,31.0
C_cv,669.0,2.029895,1.938192,1.0,1.0,1.0,2.0,15.0


,count,mean,std,min,25%,50%,75%,max
D_ca,103733.0,24.315994,104.268363,1.0,7.0,13.0,24.0,14761.0
D_pd,102891.0,10.201961,10.485127,1.0,5.0,8.0,12.0,1530.0
D_cv,383.0,2.039164,2.765917,1.0,1.0,1.0,2.0,39.0


## 人気商品は存在するのか？

人気商品が存在すれば、推薦の際に人気商品を推薦するという方法が可能になる

### 購入だけで見ると

* A(人材),B(旅行)ではC(不動産),D(アパレル)と比較して購入に関して人気商品と呼べるようなものが存在する。（以下２つ理由）
    * ABCD共通してほとんどの商品は購入されていないもしくは1,2個のみ購入されている。
    * 上記のカテゴリーごとの購入数をみると、Aは74561個、Bは21710個である。
* C,Dではまず商品数に対し購入されたことのある商品が圧倒的に少ない。(A:半数以上, B:3,4割, C:1%以下, D:0.1%以下)  
ただし、人気が集中しているわけではなく全体の購入数が少ないだけ。

In [79]:
display_df = pd.DataFrame()
for data in data_list:
    df = data["df"]
    df = df[df["event_type"] == 3]
    count_df = pd.DataFrame(df.groupby("product_id").size(), columns=[data["label"] + "_cv"])
    if display_df.empty:
        display_df = count_df.describe().T
    else:
        display_df = pd.concat([display_df, count_df.describe().T])
display(display_df)

,count,mean,std,min,25%,50%,75%,max
A_cv,9235.0,8.073741,15.786358,1.0,2.0,3.0,8.0,321.0
B_cv,6218.0,3.491476,4.354155,1.0,1.0,2.0,4.0,124.0
C_cv,862.0,1.575406,1.494317,1.0,1.0,1.0,2.0,14.0
D_cv,687.0,1.136827,0.513858,1.0,1.0,1.0,1.0,6.0


### その他も含めて見ると

* C,Dも閲覧だけで見ると人気商品と言える商品が存在する。  

**次のアクション：人気商品の洗い出しと、可視化。**
* ユーザーを人気商品を元にグループに分解できるのではないか？

In [86]:
display_df = pd.DataFrame()
events = ["ca","pd","cl","cv"]
for data in data_list:
    for i, event in enumerate(events):
        if ((data["label"] == "C") and (event == "ca")) or ((data["label"] == "D") and (event == "cl")):
            continue
        df = data["df"]
        df = df[df["event_type"] == i]
        count_df = pd.DataFrame(df.groupby("product_id").size(), columns=[data["label"] + "_" + event])
        if display_df.empty:
            display_df = count_df.describe().T
        else:
            display_df = pd.concat([display_df, count_df.describe().T])
    display(display_df)
    display_df = pd.DataFrame()
#countはその行動経験のある商品数を示している。

,count,mean,std,min,25%,50%,75%,max
A_ca,12483.0,103.079228,216.868642,1.0,11.0,37.0,109.0,4504.0
A_pd,13752.0,144.108493,282.260652,1.0,26.0,64.0,152.0,6818.0
A_cl,5796.0,5.765010,13.289840,1.0,1.0,2.0,5.0,344.0
A_cv,9235.0,8.073741,15.786358,1.0,2.0,3.0,8.0,321.0


,count,mean,std,min,25%,50%,75%,max
B_ca,7195.0,7.316053,9.419068,1.0,2.0,4.0,9.0,141.0
B_pd,16894.0,61.302060,88.473527,1.0,10.0,31.0,76.0,1421.0
B_cl,1799.0,1.911618,5.236293,1.0,1.0,1.0,2.0,143.0
B_cv,6218.0,3.491476,4.354155,1.0,1.0,2.0,4.0,124.0


,count,mean,std,min,25%,50%,75%,max
C_pd,313573.0,5.263916,8.803016,1.0,1.0,3.0,6.0,2090.0
C_cl,8394.0,1.747677,15.601379,1.0,1.0,1.0,1.0,1230.0
C_cv,862.0,1.575406,1.494317,1.0,1.0,1.0,2.0,14.0


,count,mean,std,min,25%,50%,75%,max
D_ca,638303.0,3.951683,10.481953,1.0,1.0,2.0,4.0,1015.0
D_pd,348518.0,3.011867,6.848851,1.0,1.0,1.0,3.0,516.0
D_cv,687.0,1.136827,0.513858,1.0,1.0,1.0,1.0,6.0
